In [1]:
# the below gets
# in df, all the t20 wc matches
# in df all the links of  the t20 wc matches
# for each player the batting and bowling stats

In [2]:
# import the necessary libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# the url gives the individual match details for the entire t20 wc
t20_wc_matches_URL = "https://www.espncricinfo.com/records/tournament/team-match-results/icc-men-s-t20-world-cup-2022-23-14450"
# header is used to mimic a real user
header = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36', 'Accept-Language' : 'eng-US'
            })

In [4]:
 # the below gives the table of all t20 wc matches

# access the page with the soup object
webpage = requests.get(t20_wc_matches_URL, headers=header)
t20_wc_matches_soup = BeautifulSoup(webpage.content, 'html.parser')

# put all the tables in the page in a variable
t20_wc_matches_tables = t20_wc_matches_soup.find_all('table')
# print(len(t20_wc_matches_tables))

# read the table and convert to a df
t20_wc_matches_df = pd.read_html(str(t20_wc_matches_tables), header=0)[0]
# print(t20_wc_matches_df)

# ______________________________________________________________________________
# export the df to csv
t20_wc_matches_df.to_csv('t20_wc_matches_list.csv', index=False)
#store t20_wc_matches_df

In [5]:
match_ids_dict = {}

for index, row in t20_wc_matches_df.iterrows():
    key1 = row['Team 1'] + ' vs ' + row['Team 2']
    key2 = row['Team 2'] + ' vs ' + row['Team 1']
    
    match_ids_dict[key1] = row['Scorecard']
    match_ids_dict[key2] = row['Scorecard']
    
# match_ids_dict
    

In [6]:
print(t20_wc_matches_df.iloc[1,6])
print(t20_wc_matches_df.shape)

T20I # 1878
(42, 7)


In [7]:
# the below extracts the individual team match links in the html table
# define a list to store the links in the table
all_links = []
# used to iterate iver the table got
length_row = len(t20_wc_matches_df)
# length_row = 1
# this is used to add the incomplete links got from reading the html table
string_to_be_added = "https://www.espncricinfo.com"
i=0
# print(len(t20_wc_matches_tables))
    
while(i<length_row):
    #print('hello')
    for t20_wc_matches_table in t20_wc_matches_tables:
        # Find all rows in the current table
        wc_matches_rows = t20_wc_matches_table.find_all('tr')
        #match_cells = wc_matches_rows.find_all(['td'])[1:]
        
        
        # Iterate through each element in wc_matches_rows
        for row in wc_matches_rows:
        # Find all <td> elements within the current element
            match_cells = row.find_all('td')[1:]

            # the last cell in the table has the link to individual nmatches, so go to the last cell
            last_cell = match_cells[-1]
#             find the a tag in the last cell
            link = last_cell.find('a')
#             if a tag is present, get the href attribute
            if link:
                link_href = link.get('href')
#                 added the string so that we ge the complete url
                link = string_to_be_added + link_href
#                 append the link to the a list that needs to be converted to a df
                all_links.append(link)
#                     print(f"Link in the last cell: {link}")

    i = i+1
# convert the links list got above to a df
all_links_df = pd.DataFrame(all_links, columns = ['links'])
# Keep the first occurrence and remove subsequent duplicates
all_links_df = all_links_df.drop_duplicates(keep='first')
#print(all_links_df.head())
# _______________________________________________________________________________
# export to csv
# all_links_df.to_csv('all_links_df.csv', index=False)
# %store all_links_df

In [8]:
def get_table_data(fi_si_innings_batting_bowling_table):
    fi_si_innings_batting_bowling_table_data = []
#     fi_si_batting_bowling_soup = BeautifulSoup(webpage.content, 'html.parser')
#     fi_si_batting_bowling_tables = fi_si_batting_bowling_soup.find_all('table')
    for fi_si_innings_batting_bowling_row in fi_si_innings_batting_bowling_table.find_all('tr'):
        # Extract data from the cells in the row
        cells = fi_si_innings_batting_bowling_row.find_all('td')
        # Extract the data from the cells and print it
        fi_si_innings_batting_bowling_table_data.append([cell.get_text() for cell in cells])
    return fi_si_innings_batting_bowling_table_data

In [9]:
def assign_team_names(temp_df, team_1, team_2):
    temp_df['team1'] = team_1
    temp_df['team2'] = team_2
    return temp_df

In [10]:
def convert_df_csv(df, name):
    df.to_csv(name +'.csv', index=True)
    

In [11]:
def get_team_name(batting_bowling_soup):
    #     need to add the team here
    teams = batting_bowling_soup.find_all('span', class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
    team_1 = teams[0].get_text()
    team_2 = teams[1].get_text()
    
    return team_1, team_2




In [12]:
# def concat_df(first_innings_batting_df, temp_df):
def concat_df(fi_si_innings_df, temp_df):
    fi_si_innings_df = pd.concat([fi_si_innings_df, temp_df], ignore_index=True)
    return fi_si_innings_df

In [13]:
def convert_batting_data_df(table):
    batting_temp_df = pd.DataFrame(table, columns=['BATTING', '', 'R', 'B', 'M', '4s', '6s', 'SR'])
    batting_temp_df = batting_temp_df.dropna()
    return batting_temp_df


    
    
    

In [14]:
def convert_bowling_data_df(table):
    bowling_temp_df = pd.DataFrame(table, columns=['BOWLING', 'O', 'M', 'R', 'W', 'ECON', '0s', '4s', '6s', 'WD', 'NB'])
    bowling_temp_df = bowling_temp_df.dropna()
    return bowling_temp_df

In [15]:
def get_all_tables_inAPage(team_URL, header):
    
    individual_team_webpage = requests.get(team_URL, headers=header)
    fi_si_batting_bowling_soup = BeautifulSoup(individual_team_webpage.content, 'html.parser')
    fi_si_batting_bowling_tables = fi_si_batting_bowling_soup.find_all('table')
    
    return fi_si_batting_bowling_tables

In [16]:
def remove_spl_char(df):
    
    def encode(text):
#         return text.encode('ascii', 'ignore').decode('ascii')
        return str(text).encode('ascii', 'ignore').decode('ascii')

    def decode(text):
        return text

    # Encode the entire DataFrame
    encoded_df = df.applymap(encode)

    # Decode the entire DataFrame
    decoded_df = encoded_df.applymap(decode)

    return decoded_df  


In [17]:
def combine_df(first_df, second_df):
    combined_df = pd.concat([first_df, second_df])
    return combined_df

In [18]:
# combine the two colums team1 and team2
# get the data of the two columns
# put the test 'vs' between the data of the two columns
# make this into a a new column

def combine_columns(column1, column2, df):
    data1 = df[column1]
    data2 = df[column2]

    # Combine the data with 'vs' in between
    teams_playing = data1.astype(str) + ' vs ' + data2.astype(str)

    # Add the combined data to the DataFrame
    df['teams_playing'] = teams_playing
    
    return df
    


In [19]:
def replace_text_out(df):
    df[''] = df.applymap(lambda x: 'out' if str(x).strip() != '' and 'not out' not in str(x) else x)['']
    return df

In [20]:
# the below will extract the batting and bowling statistics of each match
# find the data - first innings and second innings batting and bowling data of the teams for all matches
# find the first table (which will have first innings batting data) - with the class name - 'ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table'
# find the next table with the same class name which will have the second innings batting data
# find the first table (which will have first innings bowling data) with the class name - 'ds-w-full ds-table ds-table-md ds-table-auto'
# find the next table with the same class name which will have the second innings bowling data


# Initialize a list to store the table data of fi and si



first_innings_batting_df = pd.DataFrame()
first_innings_bowling_df = pd.DataFrame()

second_innings_batting_df = pd.DataFrame()
second_innings_bowling_df = pd.DataFrame()

combined_batting_df = pd.DataFrame()
combined_bowling_df = pd.DataFrame()

fi_si_innings_batting_bowling_table_data = []


# counter to count all links
i=0
l_df = len(all_links_df)
# l_df = 3

while(i<l_df):
#     temp_df = pd.DataFrame()
#     fi_batting_temp_df = pd.DataFrame()
    
    
    fi_si_batting_bowling_tables = []
    
    first_innings_batting_table_data = []
    first_innings_bowling_table_data = []
    
    second_innings_batting_table_data = []
    second_innings_bowling_table_data = []

    
# iloc is used here to extract the links of matches - and  iterated to get links of all matches
    individual_team_URL = all_links_df.iloc[i,0]
#     print(URL)

    header = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36', 'Accept-Language' : 'eng-US'
                })
    
# the soup is used to get the html content and the table of each page 
    individual_team_webpage = requests.get(individual_team_URL, headers=header)
    fi_si_batting_bowling_soup = BeautifulSoup(individual_team_webpage.content, 'html.parser')
    fi_si_batting_bowling_tables = fi_si_batting_bowling_soup.find_all('table')
# --------------------------------------------------------------------------------------------------------------
#     get the first innings batting data

#     first innings data in table form from the tables present in the page
    first_innings_batting_table = fi_si_batting_bowling_tables[0]

#     extract the table in the form of rows
    first_innings_batting_table_data = get_table_data(first_innings_batting_table)
    
#     convert the data from the table into a df
    fi_batting_temp_df = convert_batting_data_df(first_innings_batting_table_data)
    
#   get batting position
    fi_batting_temp_df.insert(0, 'batting_position', range(1, len(fi_batting_temp_df) + 1))
    

                              
#   get the team1 and team2 names through the function
    team_1, team_2 = get_team_name(fi_si_batting_bowling_soup)

#   assign them to temp df
    fi_batting_temp_df = assign_team_names(fi_batting_temp_df, team_1, team_2)
    
#   player belongs to which country
    teams_country = fi_si_batting_bowling_soup.find_all('span', class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
    teams_country = teams_country[0].get_text()
#     print("teams_country:    ",teams_country)
#     print(len(teams_country))
    fi_batting_temp_df['teams_country'] = teams_country
#     print(fi_batting_temp_df)

#   add two df - first_innings_batting_df (which will have all the data) and
#   temp_df (which will have a individual team data)
    first_innings_batting_df = concat_df(first_innings_batting_df, fi_batting_temp_df)

# ----------------------------------------------------------------------------

#   get the first innings bowling data

    first_innings_bowling_table = fi_si_batting_bowling_tables[1]
    first_innings_bowling_table_data = get_table_data(first_innings_bowling_table)
    
#   convert the data from the table into a df
    fi_bowling_temp_df = convert_bowling_data_df(first_innings_bowling_table_data)

#   get the team1 and team2 names through the function
    team_1, team_2 = get_team_name(fi_si_batting_bowling_soup)

#   assign them to temp df
    fi_bowling_temp_df = assign_team_names(fi_bowling_temp_df, team_1, team_2)

#   add two df - first_innings_batting_df (which will have all the data) and
#   temp_df (which will have a individual team data)
    first_innings_bowling_df = concat_df(first_innings_bowling_df, fi_bowling_temp_df)
    
#     ----------------------------------------------------------------------------------------------------

#   get the second innings batting data
    target_table = fi_si_batting_bowling_soup.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto')

#   find the next table
    next_table = target_table.find_next('table', class_='ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table')
    
#   assign the next table as second_innings_batting_table
    second_innings_batting_table = next_table
    
#   extract the table in the form of rows
    second_innings_batting_table_data = get_table_data(second_innings_batting_table) 

#   convert the data from the table into a df
    si_batting_temp_df = convert_batting_data_df(second_innings_batting_table_data)
    
#   get batting position
    si_batting_temp_df.insert(0, 'batting_position', range(1, len(si_batting_temp_df) + 1))
    
#   get the team1 and team2 names through the function
    team_1, team_2 = get_team_name(fi_si_batting_bowling_soup)
    
#   assign them to temp df
    si_batting_temp_df = assign_team_names(si_batting_temp_df, team_1, team_2)
    
#   player belongs to which country
    si_teams_country = fi_si_batting_bowling_soup.find_all('span', class_='ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate')
    si_teams_country = si_teams_country[0].get_text()
#     print("teams_country:    ",teams_country)
#     print(len(teams_country))
    si_batting_temp_df['teams_country'] = si_teams_country
#     print(si_batting_temp_df)
    
#   add two df - first_innings_batting_df (which will have all the data) and
#   temp_df (which will have a individual team data)
    second_innings_batting_df = concat_df(second_innings_batting_df, si_batting_temp_df)
    
# ---------------------------------------------------------------------------------------------

#   get the second innings bowling data

    target_table = fi_si_batting_bowling_soup.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto')

    next_table = target_table.find_next('table', class_='ds-w-full ds-table ds-table-md ds-table-auto')
    second_innings_bowling_table = next_table

    second_innings_bowling_table_data = get_table_data(second_innings_bowling_table)
    
#   convert the data from the table into a df
    si_bowling_temp_df = convert_bowling_data_df(second_innings_bowling_table_data)
    
#   get the team1 and team2 names through the function
    team_1, team_2 = get_team_name(fi_si_batting_bowling_soup)
    
#   assign them to temp df
    si_bowling_temp_df = assign_team_names(si_bowling_temp_df, team_1, team_2)
    
#   add two df - first_innings_batting_df (which will have all the data) and
#   temp_df (which will have a individual team data)
    second_innings_bowling_df = concat_df(second_innings_bowling_df, si_bowling_temp_df)

#     -----------------------------------------------------------------------------------------------

    i +=1

# # # ---------------------------------------------------------------------------------------------------------------------


# remove the special characters
first_innings_batting_df = remove_spl_char(first_innings_batting_df)

first_innings_bowling_df = remove_spl_char(first_innings_bowling_df)

second_innings_batting_df = remove_spl_char(second_innings_batting_df)

second_innings_bowling_df = remove_spl_char(second_innings_bowling_df)

# combine batting df
combined_batting_df = combine_df(first_innings_batting_df, second_innings_batting_df)
combined_bowling_df = combine_df(first_innings_bowling_df, second_innings_bowling_df)

# combine two colums - team1 and team2 - under a new column 'teams_playing'
combined_batting_df = combine_columns('team1','team2', combined_batting_df)
combined_bowling_df = combine_columns('team1','team2', combined_bowling_df)

# dropping the columns 'team1', 'team2' as they are not required
combined_batting_df.drop(columns=['team1', 'team2'], inplace=True)
combined_bowling_df.drop(columns=['team1', 'team2'], inplace=True)

# replace any text in the column with no name with the text 'out' 
# without any text in column as 'notout'
# so that the column contains only 'out' and 'not out' text
combined_batting_df = replace_text_out(combined_batting_df)

# renaming the column without a name to dismissal which will have 'out and 'not out' as row values
combined_batting_df.rename(columns={'': 'dismissal'}, inplace=True)
combined_bowling_df.rename(columns={'': 'dismissal'}, inplace=True)

# create a new column to be used as the index
combined_batting_df['index'] = range(len(combined_batting_df))
combined_bowling_df['index'] = range(len(combined_bowling_df))

# set the new column as the index
combined_batting_df = combined_batting_df.set_index('index')
combined_bowling_df = combined_bowling_df.set_index('index')





In [22]:
combined_batting_df['match_id'] = combined_batting_df['teams_playing'].map(match_ids_dict)
combined_bowling_df['match_id'] = combined_bowling_df['teams_playing'].map(match_ids_dict)

# print(combined_batting_df.head())
# print(combined_bowling_df.head())

# export df to csv 
combined_batting_df.to_csv('fi_si_batting_df.csv', index = True)
combined_bowling_df.to_csv('fi_si_bowling_df.csv', index = True)